<a href="https://colab.research.google.com/github/namhai03/Course_Computer_Vision/blob/main/Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/archive

/content/drive/MyDrive/archive


In [ ]:
# Down parametor từ model có sẵn
!wget --no-check-certificate \
https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
-O ./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2024-01-24 09:35:40--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.128.207, 142.251.6.207, 172.217.214.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.128.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

./inception_v3_weig 100%[===================>]  83.84M  60.1MB/s    in 1.4s    

2024-01-24 09:35:42 (60.1 MB/s) - ‘./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

from tensorflow.keras.preprocessing.image import ImageDataGenerator

pre_trained_model = InceptionV3(input_shape= (150,150,3),
                                include_top = False,
                                weights= None)
pre_trained_model.load_weights(local_weights_file)

In [ ]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_188 (Conv2D)         (None, 74, 74, 32)           864       ['input_3[0][0]']             
                                                                                                  
 batch_normalization_188 (B  (None, 74, 74, 32)           96        ['conv2d_188[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_188 (Activation  (None, 74, 74, 32)           0         ['batch_normalizati

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_282 (Conv2D)         (None, 74, 74, 32)           864       ['input_4[0][0]']             
                                                                                                  
 batch_normalization_282 (B  (None, 74, 74, 32)           96        ['conv2d_282[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_282 (Activation  (None, 74, 74, 32)           0         ['batch_normalizati

In [ ]:
# last_layer = pre_trained_model.get_layer("conv2d_261")

In [ ]:
# last_output = last_layer.output

In [ ]:
# last_output

<KerasTensor: shape=(None, 7, 7, 192) dtype=float32 (created by layer 'conv2d_261')>

In [ ]:
# from ast import Lambda
# from tensorflow.keras.optimizers import RMSprop

x = layer.Con
x = layers.Flatten()(last_output)

x = layers.Dense(1024, activation="relu")(x)

x = layers.Dropout(0.2)(x)

x = layers.Dense(1, activation = "sigmoid")(x)

model = Model(pre_trained_model.input, x)
model.compile(optimizer = RMSprop(lr= 0.0001),
              loss = "binary_crossentropy",
              metrics = ["acc"])

In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_188 (Conv2D)         (None, 74, 74, 32)           864       ['input_3[0][0]']             
                                                                                                  
 batch_normalization_188 (B  (None, 74, 74, 32)           96        ['conv2d_188[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_188 (Activation  (None, 74, 74, 32)           0         ['batch_normalization_18

In [ ]:
train_folder = "/content/drive/MyDrive/archive/train"
val_folder = "/content/drive/MyDrive/archive/val"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator(rescale=1.0/255.)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_folder,
                                                    batch_size=32,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

# Flow validation images in batches of 20 using test_datagen generator
val_generator = validation_datagen.flow_from_directory(val_folder,
                                                            batch_size=32,
                                                            class_mode='binary',
                                                            target_size=(150, 150))

In [ ]:
train_generator.class_indices

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs = 20,
)

In [ ]:
!pip install Image

In [ ]:

# Import các thư viện cần thiết
import numpy as np
from PIL import Image

image_path = "/content/9608c6aa70bdbdd994b41c1923a189dc.jpg"
# Load và tiền xử lý ảnh
img = Image.open(image_path)
img = img.resize((150,150))  # Thay thế kích thước theo mô hình của bạn
img_array = np.array(img) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
img_array = np.expand_dims(img_array, axis=0)  # Thêm chiều batch

# Dự đoán phân loại của ảnh
predictions = model.predict(img_array)

# Lấy ra class có xác suất cao nhất
predicted_class = np.argmax(predictions)

# In kết quả dự đoán
print("Dự đoán của mô hình:", predicted_class)